# BPE GPT2 

In [2]:
!pip install tiktoken

In [1]:
import tiktoken
import importlib

print("Tiktoken version", importlib.metadata.version("tiktoken"))

Tiktoken version 0.9.0


In [2]:
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
print(tokenizer._special_tokens)


{'<|endoftext|>': 50256}


In [4]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [5]:
print(tokenizer._special_tokens)


{'<|endoftext|>': 50256}


In [6]:
string = tokenizer.decode(integers)
string

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

# Input_Target pairs


In [7]:
with open("the-verdict.txt","r") as f:
    text = f.read()
    
encodeText = tokenizer.encode(text)

len(encodeText)

5145

In [8]:
context_size = 10

input = encodeText[:context_size]
target = encodeText[1 : context_size + 1]
print(input)
print("   ",target)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138]
    [367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257]


In [9]:
for i in range(1, context_size):
    input = encodeText[:i]
    output = encodeText[i:i+1]
    
    print(input , "---->", output)
    print(tokenizer.decode(input), "---->", tokenizer.decode(output))

[40] ----> [367]
I ---->  H
[40, 367] ----> [2885]
I H ----> AD
[40, 367, 2885] ----> [1464]
I HAD ---->  always
[40, 367, 2885, 1464] ----> [1807]
I HAD always ---->  thought
[40, 367, 2885, 1464, 1807] ----> [3619]
I HAD always thought ---->  Jack
[40, 367, 2885, 1464, 1807, 3619] ----> [402]
I HAD always thought Jack ---->  G
[40, 367, 2885, 1464, 1807, 3619, 402] ----> [271]
I HAD always thought Jack G ----> is
[40, 367, 2885, 1464, 1807, 3619, 402, 271] ----> [10899]
I HAD always thought Jack Gis ----> burn
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899] ----> [2138]
I HAD always thought Jack Gisburn ---->  rather


In [15]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, text, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # Tokenize the entire text 
        token_ids = tokenizer.encode(text)
        
        # Using a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i: i + max_length]
            target_chunck = token_ids[i+1: i + max_length + 1] 
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunck))
            
    def __len__(self):
        return len(self.input_ids)
                    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [12]:
def create_dataloader(text, max_length=256, stride=128,
                     shuffle=True, drop_last=True, num_workers=0, batch_size =4):
    
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding('gpt2')
    
    dataset = GPTDatasetV1(text, max_length, stride)
    
    dataloader = DataLoader(
        dataset = dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        num_workers = num_workers
    )
    
    return dataloader

In [16]:
dataloader = create_dataloader(text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
